<a href="https://colab.research.google.com/github/gitrekm/Coursera_Capstone/blob/master/Capstone_project_final.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#IBM Applied Data Science Capstone Course by Coursera

##Finding the best neighborhood in Paris for a popular chocolate franchise 

### 1. Building a dataframe of neighborhoods and their coordinates and population in Paris, France 

###2. Extracting the venue data for the neighborhoods from Foursquare API

###3. Explore and cluster the neighborhoods

###4. Selecting the best cluster for the new chocolate franchise

### Data Sources :

1- opendata portal : https://opendata.paris.fr/explore/dataset/quartier_paris/table/?location=12,48.85252,2.377&basemap=jawg.streets

2- Foursquare API

3- public open data soft :https://public.opendatasoft.com/explore/dataset/iris-demographie/table/

### Import Libs

In [0]:
!pip install folium
import numpy as np 
import pandas as pd 
pd.set_option("display.max_columns", None)
pd.set_option("display.max_rows", None)
import json 
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values
import requests 
# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors
# import k-means 
from sklearn.cluster import KMeans
import folium # map rendering 

In [0]:
# Download Datasets

!wget -O iris-demographie.csv https://raw.githubusercontent.com/gitrekm/Coursera_Capstone/master/data/iris-demographie.csv
!wget -O quartier_paris.csv https://raw.githubusercontent.com/gitrekm/Coursera_Capstone/master/data/quartier_paris.csv

--2019-03-28 08:46:23--  https://raw.githubusercontent.com/gitrekm/Coursera_Capstone/master/data/iris-demographie.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 9433262 (9.0M) [text/plain]
Saving to: ‘iris-demographie.csv’

iris-demographie.cs 100%[===================>]   9.00M  --.-KB/s    in 0.07s   

2019-03-28 08:46:23 (120 MB/s) - ‘iris-demographie.csv’ saved [9433262/9433262]

--2019-03-28 08:46:25--  https://raw.githubusercontent.com/gitrekm/Coursera_Capstone/master/data/quartier_paris.csv
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 151.101.0.133, 151.101.64.133, 151.101.128.133, ...
Connecting to raw.githubusercontent.com (raw.githubusercontent.com)|151.101.0.133|:443... connected.
HTTP request sent, awaiting response... 200 OK

In [0]:
#Read the first dataset 
df1 = pd.read_csv("iris-demographie.csv",delimiter=";")
# Filter only on Paris Area
df1_filtered=df1[df1['Nom Ept'] == 'Paris']
# define the dataframe columns
col_names = ['C_QU','Population'] 
# instantiate the dataframe
dd2 = pd.DataFrame(columns=col_names)
dd2.columns
dd2=df1_filtered[['N Qu','P12 Pop']].groupby('N Qu')['P12 Pop'].sum().reset_index()
dd2.head()


,N Qu,P12 Pop
0,1.0,1478.967053
1,2.0,9372.013491
2,3.0,3423.838794
3,4.0,2825.180662
4,5.0,1499.569355


In [0]:
#Read the Second dataset 
df2 = pd.read_csv("quartier_paris.csv",delimiter=";")
df2.head()

,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
0,750000010,10,7510302,Enfants-Rouges,3,750000003,2139.625388,271750.323937,"48.863887392, 2.36312330099","{""type"": ""Polygon"", ""coordinates"": [[[2.367101...",50,2139.535591
1,750000016,16,7510404,Notre-Dame,4,750000004,3283.163371,378252.153674,"48.8528955862, 2.35277501212","{""type"": ""Polygon"", ""coordinates"": [[[2.361313...",56,3282.999717
2,750000018,18,7510502,Jardin-des-Plantes,5,750000005,4052.729521,798389.398463,"48.8419401934, 2.35689388962","{""type"": ""Polygon"", ""coordinates"": [[[2.364561...",58,4052.473226
3,750000025,25,7510701,Saint-Thomas-d'Aquin,7,750000007,3827.253353,826559.436780,"48.8552632694, 2.32558765258","{""type"": ""Polygon"", ""coordinates"": [[[2.322133...",7,3827.053421
4,750000035,35,7510903,Faubourg-Montmartre,9,750000009,2786.541926,417335.080621,"48.8739346918, 2.34325257947","{""type"": ""Polygon"", ""coordinates"": [[[2.340255...",17,2786.448978


In [0]:
#Merge the two datasets we handled earlier 
df3 = pd.merge(dd2, df2, left_on="N Qu", right_on="C_QU")
df3.head()

,N Qu,P12 Pop,N_SQ_QU,C_QU,C_QUINSEE,L_QU,C_AR,N_SQ_AR,PERIMETRE,SURFACE,Geometry X Y,Geometry,OBJECTID,LONGUEUR
0,1,1478.967053,750000001,1,7510101,St-Germain-l'Auxerrois,1,750000001,5057.549475,869000.664564,"48.8606501352, 2.33491032928","{""type"": ""Polygon"", ""coordinates"": [[[2.344593...",41,5057.332772
1,2,9372.013491,750000002,2,7510102,Halles,1,750000001,2606.417128,412458.496330,"48.8622891081, 2.34489885831","{""type"": ""Polygon"", ""coordinates"": [[[2.349365...",42,2606.304934
2,3,3423.838794,750000003,3,7510103,Palais-Royal,1,750000001,2166.839239,273696.793301,"48.8646599781, 2.33630891897","{""type"": ""Polygon"", ""coordinates"": [[[2.339465...",43,2166.750303
3,4,2825.180662,750000004,4,7510104,Place-Vendôme,1,750000001,2147.817602,269456.780599,"48.8670185906, 2.32858166493","{""type"": ""Polygon"", ""coordinates"": [[[2.331944...",44,2147.734559
4,5,1499.569355,750000005,5,7510201,Gaillon,2,750000002,1866.982041,188012.203850,"48.8693066381, 2.33343180766","{""type"": ""Polygon"", ""coordinates"": [[[2.336320...",45,1866.913442


In [0]:
#Create a new dataframe with only required columns
# define the dataframe columns
column_names = ['Neighbourhood','Latitude', 'Longitude','Population'] 

# instantiate the dataframe
neighbourhoods = pd.DataFrame(columns=column_names)
neighbourhoods

,Neighbourhood,Latitude,Longitude,Population


In [0]:
# Then Instantiate it
neighbourhoods["Neighbourhood"]=df3["L_QU"]
lnglat=df3["Geometry X Y"].str.split(",", n = 1, expand = True) 
neighbourhoods["Latitude"]=lnglat[0].apply(pd.to_numeric, errors='coerce')
neighbourhoods["Longitude"]=lnglat[1].apply(pd.to_numeric, errors='coerce')
neighbourhoods["Population"]=df3["P12 Pop"]
neighbourhoods.head()

,Neighbourhood,Latitude,Longitude,Population
0,St-Germain-l'Auxerrois,48.860650,2.334910,1478.967053
1,Halles,48.862289,2.344899,9372.013491
2,Palais-Royal,48.864660,2.336309,3423.838794
3,Place-Vendôme,48.867019,2.328582,2825.180662
4,Gaillon,48.869307,2.333432,1499.569355


In [0]:
neighbourhoods.head()

,Neighbourhood,Latitude,Longitude,Population
0,St-Germain-l'Auxerrois,48.860650,2.334910,1478.967053
1,Halles,48.862289,2.344899,9372.013491
2,Palais-Royal,48.864660,2.336309,3423.838794
3,Place-Vendôme,48.867019,2.328582,2825.180662
4,Gaillon,48.869307,2.333432,1499.569355


###Create a map of Paris with neighborhoods 

In [0]:
# get the coordinates of Paris
address = 'Paris, France'
geolocator = Nominatim(user_agent="paris-app-capstone")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Paris, France {}, {}.'.format(latitude, longitude))


The geograpical coordinate of Paris, France 48.8566101, 2.3514992.


In [0]:

# create map of Paris using latitude and longitude values
map_newyork = folium.Map(location=[latitude, longitude], zoom_start=10)

# add markers to map
for lat, lng, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Neighbourhood']):
    label = '{}'.format(neighbourhood)
    label = folium.Popup(label, parse_html=True)
    folium.CircleMarker(
        [lat, lng],
        radius=5,
        popup=label,
        color='blue',
        fill=True,
        fill_color='#3186cc',
        fill_opacity=0.7,
        parse_html=False).add_to(map_newyork)  
    
map_newyork

###Define Foursquare Credentials and Version

In [0]:
CLIENT_ID = 'J5EJ4G0DXBXIRD4ST1G3TVJV1BQRCCASTAGWOCZIL145LS2Q' # your Foursquare ID
CLIENT_SECRET = '2J5WUEQHNUASEFPIWN1LXNQELD1CZMQDGBVKEDJQSEHCIYFE' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

Your credentails:
CLIENT_ID: J5EJ4G0DXBXIRD4ST1G3TVJV1BQRCCASTAGWOCZIL145LS2Q
CLIENT_SECRET:2J5WUEQHNUASEFPIWN1LXNQELD1CZMQDGBVKEDJQSEHCIYFE


###Explore Neighborhoods

In [0]:
LIMIT = 100
radius = 500


venues = []

for lat, long, neighbourhood in zip(neighbourhoods['Latitude'], neighbourhoods['Longitude'], neighbourhoods['Neighbourhood']):
    
    # create the API request URL
    url = "https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&radius={}&limit={}".format(
        CLIENT_ID,
        CLIENT_SECRET,
        VERSION,
        lat,
        long,
        radius, 
        LIMIT)
   
        
    results = requests.get(url).json()["response"]['groups'][0]['items']
    
    # return only relevant information for each nearby venue
    for venue in results:
        venues.append((
            neighbourhood,
            lat, 
            long, 
            venue['venue']['name'], 
            venue['venue']['location']['lat'], 
            venue['venue']['location']['lng'],  
            venue['venue']['categories'][0]['name']))

In [0]:
# convert the venues list into a new DataFrame
dataframe_venues = pd.DataFrame(venues)

# define the column names
dataframe_venues.columns = ['Neighbourhood', 'Latitude', 'Longitude', 'VenueName', 'VenueLatitude', 'VenueLongitude', 'VenueCategory']

print(dataframe_venues.shape)
dataframe_venues.head()

(5579, 7)


,Neighbourhood,Latitude,Longitude,VenueName,VenueLatitude,VenueLongitude,VenueCategory
0,St-Germain-l'Auxerrois,48.86065,2.33491,Musée du Louvre,48.860847,2.336440,Art Museum
1,St-Germain-l'Auxerrois,48.86065,2.33491,La Vénus de Milo (Vénus de Milo),48.859943,2.337234,Exhibit
2,St-Germain-l'Auxerrois,48.86065,2.33491,Vestige de la Forteresse du Louvre,48.861577,2.333508,Historic Site
3,St-Germain-l'Auxerrois,48.86065,2.33491,"Pavillon des Sessions – Arts d'Afrique, d'Asie...",48.860724,2.332121,Art Museum
4,St-Germain-l'Auxerrois,48.86065,2.33491,Mona Lisa | La Joconde,48.860139,2.335337,Exhibit


In [0]:
#get list of categories of venues
dataframe_venues['VenueCategory'].unique()[:60]

array(['Art Museum', 'Exhibit', 'Historic Site', 'Shopping Mall',
       'Chocolate Shop', 'Café', 'Plaza', 'Pedestrian Plaza', 'Tea Room',
       'Italian Restaurant', 'Cosmetics Shop', 'Garden', 'Theater',
       'Bridge', 'Auditorium', 'Coffee Shop', 'Smoke Shop',
       'Clothing Store', 'Sculpture Garden', 'Hotel', 'Antique Shop',
       'Furniture / Home Store', 'Government Building', 'Bakery',
       'Cocktail Bar', 'Bar', 'French Restaurant', 'Church',
       'Udon Restaurant', 'Ramen Restaurant', 'Japanese Restaurant',
       'Fountain', 'Shoe Store', 'Bistro', 'Perfume Shop',
       'Breakfast Spot', 'Sushi Restaurant', 'Spa', 'Dessert Shop',
       'Sandwich Place', 'Cupcake Shop', 'Candy Store',
       'Korean Restaurant', 'Nightclub', 'Ice Cream Shop', 'Pizza Place',
       'Boutique', 'Arts & Crafts Store', 'Bookstore', 'Toy / Game Store',
       'Chinese Restaurant', 'Restaurant', 'Wine Bar', 'Movie Theater',
       'Noodle House', 'Vegetarian / Vegan Restaurant', 'Steak

###Analysing Neighbourhoods

In [0]:
# applying one hot encoding
Neighbourhood_onehot = pd.get_dummies(dataframe_venues[['VenueCategory']], prefix="", prefix_sep="")

# add neighborhood column to dataframe
Neighbourhood_onehot['Neighbourhood'] = dataframe_venues['Neighbourhood'] 

# move neighborhood column to the first column
fixed_columns = [Neighbourhood_onehot.columns[-1]] + list(Neighbourhood_onehot.columns[:-1])
Neighbourhood_onehot = Neighbourhood_onehot[fixed_columns]

print(Neighbourhood_onehot.shape)
Neighbourhood_onehot.head()

(5579, 299)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shandong Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restau

###group rows by neighborhood and by taking the mean of the frequency of occurrence of each category

In [0]:
Neighbourhood_grouped = Neighbourhood_onehot.groupby(["Neighbourhood"]).mean().reset_index()

print(Neighbourhood_grouped.shape)
Neighbourhood_grouped.head()

(80, 299)


,Neighbourhood,Accessories Store,Afghan Restaurant,African Restaurant,Alsatian Restaurant,American Restaurant,Antique Shop,Arcade,Arepa Restaurant,Argentinian Restaurant,Art Gallery,Art Museum,Arts & Crafts Store,Arts & Entertainment,Asian Restaurant,Auditorium,Auvergne Restaurant,BBQ Joint,Baby Store,Bagel Shop,Bakery,Bar,Basketball Court,Basque Restaurant,Bed & Breakfast,Beer Bar,Beer Garden,Beer Store,Belgian Restaurant,Big Box Store,Bike Rental / Bike Share,Bistro,Board Shop,Boat or Ferry,Bookstore,Botanical Garden,Boutique,Boxing Gym,Brasserie,Brazilian Restaurant,Breakfast Spot,Breton Restaurant,Brewery,Bridge,Bubble Tea Shop,Burger Joint,Burgundian Restaurant,Bus Stop,Butcher,Cafeteria,Café,Cajun / Creole Restaurant,Cambodian Restaurant,Camera Store,Canal,Canal Lock,Candy Store,Cantonese Restaurant,Caribbean Restaurant,Cemetery,Cheese Shop,Chinese Restaurant,Chocolate Shop,Church,Circus,Clothing Store,Cocktail Bar,Coffee Shop,Comedy Club,Comfort Food Restaurant,Comic Shop,Concert Hall,Convenience Store,Corsican Restaurant,Cosmetics Shop,Costume Shop,Creperie,Cultural Center,Cupcake Shop,Cycle Studio,Dance Studio,Deli / Bodega,Department Store,Design Studio,Dessert Shop,Diner,Discount Store,Dive Bar,Doner Restaurant,Donut Shop,Eastern European Restaurant,Electronics Store,Empanada Restaurant,English Restaurant,Ethiopian Restaurant,Exhibit,Falafel Restaurant,Farmers Market,Fast Food Restaurant,Fish & Chips Shop,Fish Market,Flea Market,Flower Shop,Fondue Restaurant,Food & Drink Shop,Food Court,Food Truck,Forest,Fountain,French Restaurant,Fried Chicken Joint,Fruit & Vegetable Store,Furniture / Home Store,Gaming Cafe,Garden,Gas Station,Gastropub,Gay Bar,General College & University,General Entertainment,Gift Shop,Gluten-free Restaurant,Gourmet Shop,Government Building,Greek Restaurant,Grocery Store,Gym,Gym / Fitness Center,Gym Pool,Hawaiian Restaurant,Health Food Store,Herbs & Spices Store,Historic Site,History Museum,Hobby Shop,Hookah Bar,Hostel,Hot Dog Joint,Hotel,Hotel Bar,Ice Cream Shop,Indian Restaurant,Indie Movie Theater,Indonesian Restaurant,Irish Pub,Israeli Restaurant,Italian Restaurant,Japanese Restaurant,Jazz Club,Jewelry Store,Jewish Restaurant,Jiangxi Restaurant,Juice Bar,Karaoke Bar,Kebab Restaurant,Kids Store,Kitchen Supply Store,Korean Restaurant,Lake,Latin American Restaurant,Leather Goods Store,Lebanese Restaurant,Lingerie Store,Liquor Store,Locksmith,Lounge,Lyonese Bouchon,Mac & Cheese Joint,Malay Restaurant,Market,Martial Arts Dojo,Mediterranean Restaurant,Memorial Site,Men's Store,Metro Station,Mexican Restaurant,Middle Eastern Restaurant,Miscellaneous Shop,Mobile Phone Shop,Modern European Restaurant,Molecular Gastronomy Restaurant,Monument / Landmark,Moroccan Restaurant,Movie Theater,Multiplex,Museum,Music Store,Music Venue,New American Restaurant,Newsstand,Nightclub,Noodle House,Office,Okonomiyaki Restaurant,Opera House,Organic Grocery,Outdoor Sculpture,Outdoors & Recreation,Paper / Office Supplies Store,Park,Pastry Shop,Pedestrian Plaza,Performing Arts Venue,Perfume Shop,Peruvian Restaurant,Pet Café,Pet Store,Pharmacy,Piano Bar,Pizza Place,Planetarium,Platform,Playground,Plaza,Poke Place,Pool,Portuguese Restaurant,Provençal Restaurant,Pub,Public Art,Racecourse,Ramen Restaurant,Record Shop,Recording Studio,Recreation Center,Rental Car Location,Resort,Restaurant,Rock Club,Roof Deck,Russian Restaurant,Salad Place,Salon / Barbershop,Sandwich Place,Savoyard Restaurant,Scandinavian Restaurant,Scenic Lookout,Science Museum,Sculpture Garden,Seafood Restaurant,Shandong Restaurant,Shanxi Restaurant,Shoe Store,Shopping Mall,Shopping Plaza,Smoke Shop,Snack Place,Soba Restaurant,Soccer Field,Soccer Stadium,Soup Place,South American Restaurant,Southern / Soul Food Restaurant,Southwestern French Restaurant,Souvenir Shop,Souvlaki Shop,Spa,Spanish Restaurant,Speakeasy,Sporting Goods Shop,Sports Bar,Sports Club,Stadium,Steakhouse,Supermarket,Sushi Restaurant,Szechuan Restaurant,Taco Place,Tailor Shop,Taiwanese Restau

###Getting data of Chocolate shop only

In [0]:
Neighbourhood_chocolate = Neighbourhood_grouped[["Neighbourhood","Chocolate Shop"]]
Neighbourhood_chocolate= Neighbourhood_chocolate.merge(neighbourhoods, on="Neighbourhood")
Neighbourhood_chocolate.head()

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population
0,Amérique,0.0,48.881638,2.395440,57522.797337
1,Archives,0.0,48.859192,2.363205,8631.109190
2,Arsenal,0.0,48.851585,2.364768,8622.491574
3,Arts-et-Metiers,0.0,48.866470,2.357083,9807.016538
4,Auteuil,0.0,48.850622,2.252277,72746.492350


###Clustering 

In [0]:
# set number of clusters
kclusters = 4

chocolate_clustering = Neighbourhood_chocolate.drop(["Neighbourhood"], 1)

# run k-means clustering
kmeans = KMeans(n_clusters=kclusters, random_state=0).fit(chocolate_clustering)

# check cluster labels generated for each row in the dataframe
kmeans.labels_[0:10]
k_means_labels = kmeans.labels_


In [0]:
# create a new dataframe that includes the cluster as well as the top 10 venues for each neighborhood.
Neighbourhood_chocolate_merged = Neighbourhood_chocolate.copy()

# add clustering labels
Neighbourhood_chocolate_merged["Cluster Labels"] = kmeans.labels_

#check the centroid values by averaging the features in each cluster.
Neighbourhood_chocolate_merged.groupby('Cluster Labels').mean()

,Chocolate Shop,Latitude,Longitude,Population
Cluster Labels,,,,
0,0.001304,48.863419,2.345024,24335.193599
1,0.001111,48.865161,2.346459,45427.428363
2,0.006043,48.860417,2.341183,7388.539355
3,0.000000,48.848361,2.345104,71284.628451


In [0]:
Neighbourhood_chocolate_merged.rename(columns={"Neighbourhood": "Neighbourhood"}, inplace=True)
Neighbourhood_chocolate_merged.head()

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
0,Amérique,0.0,48.881638,2.395440,57522.797337,1
1,Archives,0.0,48.859192,2.363205,8631.109190,2
2,Arsenal,0.0,48.851585,2.364768,8622.491574,2
3,Arts-et-Metiers,0.0,48.866470,2.357083,9807.016538,2
4,Auteuil,0.0,48.850622,2.252277,72746.492350,3


In [0]:
# sort the results by Cluster Labels
print(Neighbourhood_chocolate_merged.shape)
Neighbourhood_chocolate_merged.sort_values(["Cluster Labels"], inplace=True)
Neighbourhood_chocolate_merged

(80, 6)


,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
55,Porte-Saint-Denis,0.000000,48.873618,2.352283,16741.838902,0
31,Hôpital-Saint-Louis,0.000000,48.876008,2.368123,29521.761371,0
26,Goutte-d'Or,0.000000,48.892138,2.355536,31921.146090,0
61,Saint-Ambroise,0.000000,48.862345,2.376118,33310.807803,0
58,Quinze-Vingts,0.000000,48.846916,2.374402,27494.508169,0
23,Folie-Méricourt,0.000000,48.867403,2.372965,33602.268870,0
63,Saint-Georges,0.000000,48.879934,2.332850,22319.308056,0
33,Jardin-des-Plantes,0.000000,48.841940,2.356894,18219.038499,0
20,Europe,0.000000,48.878148,2.317175,19856.370455,0
35,La Chapelle,0.000000,48.894012,2.364387,25856.774876,0


In [0]:
#Visualize the results

# create map
map_clusters = folium.Map(location=[latitude, longitude], zoom_start=11)

# set color scheme for the clusters
x = np.arange(kclusters)
ys = [i+x+(i*x)**2 for i in range(kclusters)]
colors_array = cm.rainbow(np.linspace(0, 1, len(ys)))
rainbow = [colors.rgb2hex(i) for i in colors_array]

# add markers to the map
markers_colors = []
for lat, lon, poi, cluster in zip(Neighbourhood_chocolate_merged['Latitude'], Neighbourhood_chocolate_merged['Longitude'], Neighbourhood_chocolate_merged['Neighbourhood'], Neighbourhood_chocolate_merged['Cluster Labels']):
    label = folium.Popup(str(poi) + ' - Cluster ' + str(cluster), parse_html=True)
    folium.CircleMarker(
        [lat, lon],
        radius=5,
        popup=label,
        color=rainbow[cluster-1],
        fill=True,
        fill_color=rainbow[cluster-1],
        fill_opacity=0.7).add_to(map_clusters)
       
map_clusters

In [0]:
#Examine Cluster
Neighbourhood_chocolate_merged.loc[Neighbourhood_chocolate_merged['Cluster Labels'] == 0]

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
55,Porte-Saint-Denis,0.00,48.873618,2.352283,16741.838902,0
31,Hôpital-Saint-Louis,0.00,48.876008,2.368123,29521.761371,0
26,Goutte-d'Or,0.00,48.892138,2.355536,31921.146090,0
61,Saint-Ambroise,0.00,48.862345,2.376118,33310.807803,0
58,Quinze-Vingts,0.00,48.846916,2.374402,27494.508169,0
23,Folie-Méricourt,0.00,48.867403,2.372965,33602.268870,0
63,Saint-Georges,0.00,48.879934,2.332850,22319.308056,0
33,Jardin-des-Plantes,0.00,48.841940,2.356894,18219.038499,0
20,Europe,0.00,48.878148,2.317175,19856.370455,0
35,La Chapelle,0.00,48.894012,2.364387,25856.774876,0


In [0]:
#Examine Cluster
Neighbourhood_chocolate_merged.loc[Neighbourhood_chocolate_merged['Cluster Labels'] == 1]

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
57,Père-Lachaise,0.00,48.863719,2.395273,46367.562377,1
41,Muette,0.00,48.863275,2.259936,46367.385274,1
34,Javel 15Art,0.00,48.839060,2.278076,52650.432267,1
42,Necker,0.00,48.842711,2.310777,48647.246545,1
51,Plaine de Monceaux,0.00,48.885044,2.302910,42259.813473,1
78,Villette,0.00,48.887661,2.374468,56468.653544,1
0,Amérique,0.00,48.881638,2.395440,57522.797337,1
62,Saint-Fargeau,0.00,48.871035,2.406172,46089.165053,1
60,Roquette,0.00,48.857064,2.380364,49904.658384,1
48,Petit-Montrouge,0.00,48.826653,2.326437,39292.200669,1


In [0]:
#Examine Cluster
Neighbourhood_chocolate_merged.loc[Neighbourhood_chocolate_merged['Cluster Labels'] == 2]

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
75,St-Germain-l'Auxerrois,0.010000,48.860650,2.334910,1478.967053,2
74,Sorbonne,0.010000,48.849045,2.345747,9946.538880,2
71,Sainte-Avoie,0.000000,48.862557,2.354852,7983.605354,2
68,Saint-Thomas-d'Aquin,0.010989,48.855263,2.325588,11635.813470,2
67,Saint-Merri,0.000000,48.858521,2.351667,6171.645276,2
65,Saint-Gervais,0.000000,48.855719,2.358162,9702.923510,2
64,Saint-Germain-des-Prés,0.020000,48.855289,2.333657,4796.057084,2
69,Saint-Victor,0.000000,48.847664,2.354093,12279.079237,2
50,Place-Vendôme,0.030000,48.867019,2.328582,2825.180662,2
39,Monnaie,0.030000,48.854384,2.340035,5418.632175,2


In [0]:
#Examine Cluster
Neighbourhood_chocolate_merged.loc[Neighbourhood_chocolate_merged['Cluster Labels'] == 3]

,Neighbourhood,Chocolate Shop,Latitude,Longitude,Population,Cluster Labels
12,Charonne,0.0,48.854760,2.407430,66956.368217,3
66,Saint-Lambert,0.0,48.834294,2.296920,86940.165696,3
38,Maison-Blanche,0.0,48.823128,2.352433,68107.354970,3
4,Auteuil,0.0,48.850622,2.252277,72746.492350,3
49,Picpus,0.0,48.830359,2.428827,67377.976213,3
27,Grandes-Carrières,0.0,48.892578,2.334363,73347.891441,3
52,Plaisance,0.0,48.830317,2.315305,60211.075588,3
14,Clignancourt,0.0,48.891668,2.345979,70248.187593,3
25,Gare,0.0,48.827527,2.372398,75626.143993,3
